In [90]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image

from model import build_model
from load_data import get_patches, data_generate

PATCH_SHAPE = (224, 224, 3)

clinical_data = pd.read_csv("data/train.csv")

bag_names = list(clinical_data["ID"])
labels = list(clinical_data["N_category"])
patch_bags = get_patches("data/train_imgs_patch")

In [103]:
from sklearn.model_selection import train_test_split

In [105]:
train_test_split(bag_names, labels, patch_bags)

[['BC_01_3111',
  'BC_01_2489',
  'BC_01_2691',
  'BC_01_1015',
  'BC_01_1470',
  'BC_01_2198',
  'BC_01_2782',
  'BC_01_2993',
  'BC_01_2877',
  'BC_01_2390',
  'BC_01_2858',
  'BC_01_2950',
  'BC_01_0264',
  'BC_01_2349',
  'BC_01_1120',
  'BC_01_0886',
  'BC_01_0963',
  'BC_01_1418',
  'BC_01_1256',
  'BC_01_2333',
  'BC_01_1204',
  'BC_01_2699',
  'BC_01_2439',
  'BC_01_3004',
  'BC_01_2399',
  'BC_01_3324',
  'BC_01_3048',
  'BC_01_2987',
  'BC_01_3049',
  'BC_01_3032',
  'BC_01_2106',
  'BC_01_2140',
  'BC_01_1169',
  'BC_01_1509',
  'BC_01_2232',
  'BC_01_1533',
  'BC_01_2238',
  'BC_01_3051',
  'BC_01_2180',
  'BC_01_0710',
  'BC_01_1645',
  'BC_01_2195',
  'BC_01_2939',
  'BC_01_0563',
  'BC_01_2234',
  'BC_01_0508',
  'BC_01_1796',
  'BC_01_0007',
  'BC_01_2866',
  'BC_01_1952',
  'BC_01_0893',
  'BC_01_0304',
  'BC_01_3337',
  'BC_01_1881',
  'BC_01_0423',
  'BC_01_2744',
  'BC_01_1495',
  'BC_01_2978',
  'BC_01_2660',
  'BC_01_0737',
  'BC_01_1326',
  'BC_01_3502',
  'BC_01

In [106]:
train_test_split(bag_names, labels, patch_bags)

[['BC_01_2222',
  'BC_01_1637',
  'BC_01_0765',
  'BC_01_0503',
  'BC_01_0637',
  'BC_01_0693',
  'BC_01_3518',
  'BC_01_3058',
  'BC_01_0893',
  'BC_01_2674',
  'BC_01_3017',
  'BC_01_0736',
  'BC_01_3001',
  'BC_01_2584',
  'BC_01_2269',
  'BC_01_1161',
  'BC_01_1308',
  'BC_01_2813',
  'BC_01_1314',
  'BC_01_1204',
  'BC_01_1557',
  'BC_01_3030',
  'BC_01_0992',
  'BC_01_2708',
  'BC_01_1181',
  'BC_01_2031',
  'BC_01_2985',
  'BC_01_2084',
  'BC_01_3039',
  'BC_01_3208',
  'BC_01_1962',
  'BC_01_3267',
  'BC_01_2101',
  'BC_01_1253',
  'BC_01_1875',
  'BC_01_3111',
  'BC_01_1276',
  'BC_01_2968',
  'BC_01_2959',
  'BC_01_2691',
  'BC_01_3004',
  'BC_01_2734',
  'BC_01_2960',
  'BC_01_2881',
  'BC_01_3179',
  'BC_01_2890',
  'BC_01_0710',
  'BC_01_2024',
  'BC_01_3248',
  'BC_01_0264',
  'BC_01_2185',
  'BC_01_1101',
  'BC_01_3502',
  'BC_01_0886',
  'BC_01_1190',
  'BC_01_0352',
  'BC_01_0492',
  'BC_01_0788',
  'BC_01_2267',
  'BC_01_2966',
  'BC_01_1859',
  'BC_01_0510',
  'BC_01

In [107]:
train_bag_names, val_bag_names, train_y, val_y, train_bags, val_bags = train_test_split(bag_names, labels, patch_bags)

In [93]:
train_dataset = tf.data.Dataset.from_generator(
    generator=data_generate,
    output_types=(tf.float32, tf.float32),
    output_shapes=(tf.TensorShape([224, 224, 3]), tf.TensorShape([])),
    args=(train_bag_names, train_y, train_bags)
)

val_dataset = tf.data.Dataset.from_generator(
    generator=data_generate,
    output_types=(tf.float32, tf.float32),
    output_shapes=(tf.TensorShape([224, 224, 3]), tf.TensorShape([])),
    args=(val_bag_names, test_y, test_bags)
)


train_dataset = train_dataset.batch(100)

In [95]:
model = build_model(PATCH_SHAPE)
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 221, 221, 36  1764        ['input[0][0]']                  
                                )                                                                 
                                                                                                  
 max_pooling2d_18 (MaxPooling2D  (None, 110, 110, 36  0          ['conv2d_19[0][0]']              
 )                              )                                                           

In [96]:
from loss import bag_loss, bag_accuracy
model.compile(
    optimizer=tf.keras.optimizers.Adam(), loss=bag_loss, metrics=[bag_accuracy]
)

In [101]:
dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [100]:
model()

ValueError: The first argument to `Layer.call` must always be passed.

In [102]:
model.fit(
    dataset,
    validation_split=0.2
)

ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>]